In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

In [3]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

In [4]:
# credentials

consumer_key = "K5r6LL6m7cUTD1oEWiM0aEked"
consumer_secret = "JvSFSbTpK1yDoTMuPVnBb43mpF55Ibn7kVVvCpeTZZe2p4H616"
access_token = "1452986772154040322-avpgLnK5tHKqiqFNNNQ3ULBnELunDl"
access_token_secret = "9Y6i3eV6MWak7mn34aYIfLlLU1knc9uSHO3ZyERUALAhi"

In [5]:
#call API

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)


In [6]:
#providing query which i need to search'

query = "COVID-19"

#fetch tweets

Tweets = api.search_tweets(query , count = 10 , lang = 'en' , exclude = 'retweets' , tweet_mode = 'extended')

for tweet in Tweets:
    print(tweet)
    print('===================================================================================')

Unexpected parameter: exclude


Status(_api=<tweepy.api.API object at 0x0000020D0115BBE0>, _json={'created_at': 'Thu Jul 14 16:37:50 +0000 2022', 'id': 1547621401011490823, 'id_str': '1547621401011490823', 'full_text': 'COVID-19 Public Emergency to Continue, Hospices in the Dark on Regulatory Future.\nhttps://t.co/SGIzFTThpx', 'truncated': False, 'display_text_range': [0, 105], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/SGIzFTThpx', 'expanded_url': 'https://hospicenews.com/2022/07/08/covid-19-public-emergency-likely-to-continue-hospices-remain-in-the-dark-on-regulatory-future/?itm_source=parsely-api', 'display_url': 'hospicenews.com/2022/07/08/cov…', 'indices': [82, 105]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_scr

# Gettting the tweets and some of its related attributes

In [7]:
df = pd.DataFrame(columns= ['Tweets','User','User_statuses_count','user_followers','user_location','user_verified','fav_count'
                            ,'retweet_count','tweet_date'])
df

,Tweets,User,User_statuses_count,user_followers,user_location,user_verified,fav_count,retweet_count,tweet_date


In [8]:
def stream(data,file_name):
    i=0
    for tweet in tweepy.Cursor(api.search_tweets, q=data,count = 100 , lang='en').items():
        print(i,end='\r')
        df.loc[i , 'Tweets'] = tweet.text
        df.loc[i , 'User'] = tweet.user.name
        df.loc[i , 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i , 'user_followers'] = tweet.user.followers_count
        df.loc[i , 'user_location'] = tweet.user.location
        df.loc[i , 'user_verified'] = tweet.user.verified
        df.loc[i , 'fav_count'] = tweet.favorite_count
        df.loc[i , 'retweet_count'] = tweet.retweet_count
        df.loc[i , 'tweet_date'] = tweet.created_at
        df['tweet_date'] = df['tweet_date'].apply(lambda a: pd.to_datetime(a).date())
        df.to_excel('{}.xlsx'.format(file_name))
        i=i+1
        if i ==1000:
            break
        else:
            pass
    

In [10]:
stream(data=["COVID-19"],file_name='my_tweets')


In [11]:
df.head()

,Tweets,User,User_statuses_count,user_followers,user_location,user_verified,fav_count,retweet_count,tweet_date
0,"RT @AP: Bill Gates, concerned about the ""signi...","Rev. Isaac ""Workers United"" Collins",605,92,,False,0,90,2022-07-14
1,RT @drcmday: I am a doctor on the frontline in...,Jocky,1882,45,,False,0,2152,2022-07-14
2,RT @Togetherdec: Parliament is going to debate...,Stephan Hollemans,215848,842,"The Hague, The Netherlands",False,0,135,2022-07-14
3,"OSDH: 10,309 More COVID-19 Cases Reported Stat...",News 9,153801,203516,Oklahoma City,True,0,0,2022-07-14
4,RT @CiplaUganda: general health checkups inclu...,Mulungi Faith Liam,403967,22934,"Kampala,Uganda",False,0,5,2022-07-14


In [12]:
df.shape

(1000, 9)

In [13]:
from textblob import TextBlob

In [14]:
#Textblob has built in sentiment property

#Sentiment(polarity,subjectivity)

#polarity:It is a float score within the range [-1,1]
#subjectivity:it is a float value in the range [0,1] where 0 means very objective and 1 means very subjective.


In [16]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [17]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return "Positive"
    elif analysis.sentiment.polarity==0:
        return "Neutral"
    else:
        return "Negative"

In [18]:
df['clean_tweet'] = df["Tweets"].apply(lambda x : clean_tweet(x))
df[['Tweets','clean_tweet']].head()

,Tweets,clean_tweet
0,"RT @AP: Bill Gates, concerned about the ""signi...",RT Bill Gates concerned about the significant ...
1,RT @drcmday: I am a doctor on the frontline in...,RT I am a doctor on the frontline in A amp E t...
2,RT @Togetherdec: Parliament is going to debate...,RT Parliament is going to debate the petition ...
3,"OSDH: 10,309 More COVID-19 Cases Reported Stat...",OSDH 10 309 More COVID 19 Cases Reported State...
4,RT @CiplaUganda: general health checkups inclu...,RT general health checkups including optical a...


In [19]:
df["Sentiment"] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x))

In [20]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT Bill Gates concerned about the significant ...,Positive
1,RT I am a doctor on the frontline in A amp E t...,Negative
2,RT Parliament is going to debate the petition ...,Neutral
3,OSDH 10 309 More COVID 19 Cases Reported State...,Positive
4,RT general health checkups including optical a...,Positive


In [21]:
n = 26
print('Original Tweet:\n'+df["Tweets"][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
Digital minister tests positive for COVID-19 on visit to the U.S. | The Asahi Shimbun: Breaking News, Japan News an… https://t.co/TgKlriMuwx

Clean tweet:
Digital minister tests positive for COVID 19 on visit to the U S The Asahi Shimbun Breaking News Japan News an

Sentiment:
Positive


In [22]:
n = 88
print('Original Tweet:\n'+df["Tweets"][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
Bernadette Mazurek Melnyk, PhD, in an exclusive interview with ICT, discusses how facilities and IPs themselves can… https://t.co/AeLMVzqLwp

Clean tweet:
Bernadette Mazurek Melnyk PhD in an exclusive interview with ICT discusses how facilities and IPs themselves can

Sentiment:
Neutral


In [23]:
n = 33
print('Original Tweet:\n'+df["Tweets"][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
Fait In Action will be offering community assistance at a special event 10 a.m. to 3 p.m. on Saturday, July 16, at… https://t.co/C40JtjLgvC

Clean tweet:
Fait In Action will be offering community assistance at a special event 10 a m to 3 p m on Saturday July 16 at

Sentiment:
Positive


In [30]:
n = 70
print('Original Tweet:\n'+df["Tweets"][n])
print()
print('Clean tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @irajulpandya: As part of #AzadiKaAmritMahotsav celebrations, free COVID-19 Precaution Dose will be administered to all the citizens abo…

Clean tweet:
RT As part of AzadiKaAmritMahotsav celebrations free COVID 19 Precaution Dose will be administered to all the citizens abo

Sentiment:
Positive


In [31]:
df[df.Sentiment=='Positive'].shape[0]

385

In [32]:
df[df.Sentiment=='Negative'].shape[0]

193

In [33]:
df[df.Sentiment=='Neutral'].shape[0]

422